In [ ]:
import threading
import time

def background_task():
    while True:
        print("Tache en arriere-plan")
        time.sleep(1)


# Creer un thread en arriere-plan (daemon)
background_thread = threading.Thread(target = background_task)
background_thread.daemon = True

# Demarrer le thread en arrier-plan
background_thread.start()

print("start")
time.sleep(5)
print("end")
print("Fin du programme principale")

# Probleme 1
### Un agent globale qui doit collecter des informations provenant de trois locaux fonctionnant en parallele 

In [2]:
import threading

class GlobalAgent:
    
    def __init__(self):
        self.lock = threading.Lock()
        self.data = []
        
    def receive_data(self, agent_id, info):
        with self.lock:
            self.data.append(f"Agent {agent_id} : info")


def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f" Data {i}"
        global_agent.receive_data(agent_id, info)
        print(f"Agent {agent_id} sent : {info}")
        

In [3]:
global_agent = GlobalAgent()

agent_1 = threading.Thread(target=local_agent, args=(1, global_agent))
agent_2 = threading.Thread(target=local_agent, args=(2, global_agent))
agent_3 = threading.Thread(target=local_agent, args=(3, global_agent))

agent_1.start()
agent_2.start()
agent_3.start()

agent_1.join()
agent_2.join()
agent_3.join()

print("Donnees collectees par l'agent globale")
for data in global_agent.data:
    print(data)

Agent 1 sent :  Data 0Agent 2 sent :  Data 0
Agent 2 sent :  Data 1
Agent 2 sent :  Data 2
Agent 2 sent :  Data 3
Agent 2 sent :  Data 4

Agent 1 sent :  Data 1
Agent 1 sent :  Data 2
Agent 1 sent :  Data 3
Agent 1 sent :  Data 4
Agent 3 sent :  Data 0
Agent 3 sent :  Data 1
Agent 3 sent :  Data 2
Agent 3 sent :  Data 3
Agent 3 sent :  Data 4
Donnees collectees par l'agent globale
Agent 1 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 1 : info
Agent 1 : info
Agent 1 : info
Agent 1 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info


## - Les agents doivent communiquer entre eux, et partager les donnees avec l'agent globale

In [1]:
import threading
import queue
import time

class GlobalAgent:
    
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data = []
        self.agents = [queue.Queue() for _ in range(agent_count)]

    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id}: {info}")

    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)


def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f"Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        time.sleep(1)


if __name__ == "__main__":
    global_agent = GlobalAgent(agent_count=3)

    # Créer trois agents locaux dans leurs propres threads
    thread1 = threading.Thread(target=local_agent, args=(0, global_agent))
    thread2 = threading.Thread(target=local_agent, args=(1, global_agent))
    thread3 = threading.Thread(target=local_agent, args=(2, global_agent))

    # Démarrer les threads des agents locaux
    thread1.start()
    thread2.start()
    thread3.start()

    # Attendre que tous les threads se terminent
    thread1.join()
    thread2.join()
    thread3.join()

    # Afficher les données collectées par l'agent global
    print("Données collectées par l'agent global:")
    for data in global_agent.data:
        print(data)

    # Afficher les messages envoyés entre les agents
    print("\nMessages entre les agents:")
    for i, agent_queue in enumerate(global_agent.agents):
        while not agent_queue.empty():
            message = agent_queue.get()
            print(f"Agent {i} received: {message}")


Données collectées par l'agent global:
2023-10-11 15:35:29 - Agent 0: Data 0
2023-10-11 15:35:29 - Agent 1: Data 0
2023-10-11 15:35:29 - Agent 2: Data 0
2023-10-11 15:35:30 - Agent 0: Data 1
2023-10-11 15:35:30 - Agent 1: Data 1
2023-10-11 15:35:30 - Agent 2: Data 1
2023-10-11 15:35:31 - Agent 0: Data 2
2023-10-11 15:35:31 - Agent 1: Data 2
2023-10-11 15:35:31 - Agent 2: Data 2
2023-10-11 15:35:32 - Agent 0: Data 3
2023-10-11 15:35:32 - Agent 1: Data 3
2023-10-11 15:35:32 - Agent 2: Data 3
2023-10-11 15:35:33 - Agent 0: Data 4
2023-10-11 15:35:33 - Agent 1: Data 4
2023-10-11 15:35:33 - Agent 2: Data 4

Messages entre les agents:
Agent 0 received: Message 0
Agent 0 received: Message 1
Agent 0 received: Message 2
Agent 0 received: Message 3
Agent 0 received: Message 4
Agent 1 received: Message 0
Agent 1 received: Message 1
Agent 1 received: Message 2
Agent 1 received: Message 3
Agent 1 received: Message 4
Agent 2 received: Message 0
Agent 2 received: Message 1
Agent 2 received: Message 2

## L'agent globale doit a sont tour pouvoir communiquer avecles agents locals

In [5]:
import threading
import queue
import time


class GlobalAgent:
    
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data =[]
        self.agents = [queue.Queue() for _ in range(agent_count)]
        self.global_queue = queue.Queue()
        
    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id} : {info}")
    
    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)
    
    def send_global_message(self, message):
        self.global_queue.put(message)
    
    def receive_message(self, agent_id):
        return self.agents[agent_id].get()
    

def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f" Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        global_message = global_agent.receive_globa_message()
        print(f"Agent {agent_id} received glo")

In [1]:
import threading
import queue
import time

class GlobalAgent:
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data = []
        self.agents = [queue.Queue() for _ in range(agent_count)]
        self.global_queue = queue.Queue()

    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id}: {info}")

    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)

    def send_global_message(self, message):
        self.global_queue.put(message)

    def receive_message(self, agent_id):
        return self.agents[agent_id].get()

    def receive_global_message(self):
        return self.global_queue.get()

def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f"Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        global_message = global_agent.receive_global_message()
        print(f"Agent {agent_id} received global message: {global_message}")
        time.sleep(1)

if __name__ == "__main__":
    global_agent = GlobalAgent(agent_count=3)

    # Créer trois agents locaux dans leurs propres threads
    thread1 = threading.Thread(target=local_agent, args=(0, global_agent))
    thread2 = threading.Thread(target=local_agent, args=(1, global_agent))
    thread3 = threading.Thread(target=local_agent, args=(2, global_agent))

    # Démarrer les threads des agents locaux
    thread1.start()
    thread2.start()
    thread3.start()

    # Attendre que tous les threads se terminent
    thread1.join()
    thread2.join()
    thread3.join()

    # Afficher les données collectées par l'agent global
    print("Données collectées par l'agent global:")
    for data in global_agent.data:
        print(data)


## Utilisation de la file d'attente

In [1]:
#import queue
import multiprocessing
import threading
import time

def worker(q):
    while True:
        try:
            item = q.get(timeout=1)
            print(f"Traitement de {item}")
            q.task_done()
        except queue.Empty:
            print("La file d'attente est vide, fin du thread.")
            break


# Creer une file d'attente
#ma_file = queue.Queue()
ma_file = multiprocessing.Queue()

# Creer et demarrer deux threads de travail
agent_1 = threading.Thread(target = worker, args = (ma_file, ))
agent_2 = threading.Thread(target = worker, args = (ma_file, ))

agent_1.start()
agent_2.start()

# Mettre des elements dans la file d'attente
ma_file.put("Element 1")
ma_file.put("Element 2")
ma_file.put("Element 3")

# Attendre que les threads terminent
agent_1.join()
agent_2.join()

print("Tous les elements ont ete traites")


Exception in thread Exception in thread Thread-6 (worker):
Traceback (most recent call last):
  File "C:\Users\cc\AppData\Local\Temp\ipykernel_15640\3708340767.py", line 11, in worker
Thread-5 (worker):
Traceback (most recent call last):
  File "C:\Users\cc\AppData\Local\Temp\ipykernel_15640\3708340767.py", line 11, in worker
AttributeError: 'Queue' object has no attribute 'task_done'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\cc\miniconda3\lib\threading.py", line 1016, in _bootstrap_inner
AttributeError: 'Queue' object has no attribute 'task_done'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\cc\miniconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\cc\miniconda3\lib\threading.py", line 953, in run
    self.run()
  File "c:\Users\cc\miniconda3\lib\threading.py", line 953, in run
    self._target

Traitement de Element 1
Traitement de Element 2
Tous les elements ont ete traites


## Exemple d'utilisation de la file d'attente prioritaire

In [ ]:
import queue

# Creer une file d'attente prioritaire
ma_file_prioritaire = queue.PriorityQueue()

# Ajouter des elements avec des priorites
ma_file_prioritaire.put((2, "Element 2"))
ma_file_prioritaire.put((1, "Element 1"))
ma_file_prioritaire.put((3, "Element 3"))

while not ma_file_prioritaire.empty():
    priority, item = ma_file_prioritaire.get()
    print(f"Element de priorite {priority} : {item}")
    

## Exemple d'utilisation de la file d'attente de type Lifo (Last-In, First=Out)

In [ ]:
import queue

# Creer une file d'attente de type Lifo
ma_file_lifo = queue.LifoQueue()

# Ajouter des elements
ma_file_lifo.put("Element 1")
ma_file_lifo.put("Element 2")
ma_file_lifo.put("Element 3")

# Recuperer les elements dans l'ordre Lifo (Last-In, First-Out)
while not ma_file_lifo.empty():
    item = ma_file_lifo.get()
    print(f" Element : {item}")
    

In [ ]:
"U"

In [1]:
import multiprocessing

def agent1(condition, data):
    with condition:
        print("Agent 1: Je travaille sur mes tâches.")
        data.put("Tâche de l'Agent 1 terminée.")
        condition.notify()  # Notifie le maître qu'une tâche est terminée
        print("1 . Notifie au maitre ")
        condition.wait()    # Attend que le maître donne la permission de continuer

def agent2(condition, data):
    with condition:
        print("Agent 2: Je travaille sur mes tâches.")
        data.put("Tâche de l'Agent 2 terminée.")
        condition.notify()  # Notifie le maître qu'une tâche est terminée
        print("2 . Notifie au maitre ")
        condition.wait()    # Attend que le maître donne la permission de continuer


def master(condition, data):
    with condition:
        # Attendez que les deux agents aient terminé leur travail
        print("Attend les donnees des agents ... ")
        print(" size ",data.qsize())
        condition.wait_for(lambda: data.qsize() == 2)

        print("Maître : Les agents ont terminé leurs tâches.")
        data.put("Toutes les tâches sont terminées.")
        condition.notify_all()  # Notifie tous les agents que le travail est terminé


if __name__ == '__main__':
    data = multiprocessing.Queue()
    condition = multiprocessing.Condition()
        
    agent1_process = multiprocessing.Process(target=agent1, args=(condition, data))
    agent2_process = multiprocessing.Process(target=agent2, args=(condition, data))
    master_process = multiprocessing.Process(target=master, args=(condition, data))

    agent1_process.start()
    agent2_process.start()
    master_process.start()

    agent1_process.join()
    agent2_process.join()
    master_process.join()

    print("Résultat final:", data)


Résultat final: <multiprocessing.queues.Queue object at 0x000001A7EC329210>


In [ ]:
import multiprocessing

def producer(condition, shared_data):
    for item in range(1, 100):
        with condition:
            shared_data.put(item)
            print(f"Produit : {item}")
            condition.notify()  # Signale aux consommateurs qu'un nouvel élément est disponible
            condition.wait()  # Attend que les consommateurs finissent

def consumer(condition, shared_data):
    for _ in range(100):
        with condition:
            while shared_data.empty():
                condition.wait()  # Attendez que le producteur signale
            item = shared_data.get()
            print(f"Consommé : {item}")
            condition.notify()  # Signale au producteur que l'élément a été consommé


if __name__ == '__main__':
        
    shared_data = multiprocessing.Queue()
    condition = multiprocessing.Condition()

    producer_process = multiprocessing.Process(target=producer, args=(condition, shared_data))
    consumer_process = multiprocessing.Process(target=consumer, args=(condition, shared_data))

    producer_process.start()
    consumer_process.start()

    producer_process.join()
    consumer_process.join()


In [8]:
import multiprocessing

# Fonction pour l'agent 1
def agent1(condition, data1, data2):
    with condition:
        data1.append(1)  # Ajoutez des données à data1
        condition.notify()  # Notifie le maître
        condition.wait()  # Attendez que le maître réponde
        data2.append(2)  # Ajoutez des données à data2

# Fonction pour l'agent 2
def agent2(condition, data1, data2):
    with condition:
        data1.append(3)  # Ajoutez des données à data1
        condition.notify()  # Notifie le maître
        condition.wait()  # Attendez que le maître réponde
        data2.append(4)  # Ajoutez des données à data2

# Fonction pour le maître
def master(condition, data1, data2):
    with condition:
        condition.wait()  # Attendez l'agent 1
        print("Maître a reçu une notification de l'agent 1")
        condition.notify()  # Répondez à l'agent 1
        condition.wait()  # Attendez l'agent 2
        print("Maître a reçu une notification de l'agent 2")
        condition.notify()  # Répondez à l'agent 2

if __name__ == '__main':
    condition = multiprocessing.Condition()
    data1 = []
    data2 = []

    agent1_process = multiprocessing.Process(target=agent1, args=(condition, data1, data2))
    agent2_process = multiprocessing.Process(target=agent2, args=(condition, data1, data2))
    master_process = multiprocessing.Process(target=master, args=(condition, data1, data2))
    
    print(agent1_process.is_alive())
    
    agent1_process.start()
    agent2_process.start()
    master_process.start()
    
    print(agent1_process.is_alive())
    
    agent1_process.join()
    agent2_process.join()
    master_process.join()

    print("Données de l'agent 1:", data1)
    print("Données de l'agent 2:", data2)


False

In [8]:
import multiprocessing

def producer(condition, shared_data):
    for item in range(1, 100):
        with condition:
            shared_data.put(item)
            print(f"Produit : {item}")
            condition.notify()  # Signale aux consommateurs qu'un nouvel élément est disponible
            condition.wait()  # Attend que les consommateurs finissent

def consumer(condition, shared_data):
    for _ in range(100):
        with condition:
            while shared_data.empty():
                condition.wait()  # Attendez que le producteur signale
            item = shared_data.get()
            print(f"Consommé : {item}")
            condition.notify()  # Signale au producteur que l'élément a été consommé


if __name__ == '__main__':
        
    shared_data = multiprocessing.Queue()
    condition = multiprocessing.Condition()

    producer_process = multiprocessing.Process(target=producer, args=(condition, shared_data))
    consumer_process = multiprocessing.Process(target=consumer, args=(condition, shared_data))

    producer_process.start()
    consumer_process.start()

    producer_process.join()
    consumer_process.join()


In [1]:
import multiprocessing

# Fonction pour l'agent 1
def agent1(condition, data1, data2):
    with condition:
        data1.append(1)  # Ajoutez des données à data1
        condition.notify()  # Notifie le maître
        condition.wait()  # Attendez que le maître réponde
        data2.append(2)  # Ajoutez des données à data2

# Fonction pour l'agent 2
def agent2(condition, data1, data2):
    with condition:
        data1.append(3)  # Ajoutez des données à data1
        condition.notify()  # Notifie le maître
        condition.wait()  # Attendez que le maître réponde
        data2.append(4)  # Ajoutez des données à data2

# Fonction pour le maître
def master(condition, data1, data2):
    with condition:
        condition.wait()  # Attendez l'agent 1
        print("Maître a reçu une notification de l'agent 1")
        condition.notify()  # Répondez à l'agent 1
        condition.wait()  # Attendez l'agent 2
        print("Maître a reçu une notification de l'agent 2")
        condition.notify()  # Répondez à l'agent 2

if __name__ == '__main':
    condition = multiprocessing.Condition()
    data1 = []
    data2 = []

    agent1_process = multiprocessing.Process(target=agent1, args=(condition, data1, data2))
    agent2_process = multiprocessing.Process(target=agent2, args=(condition, data1, data2))
    master_process = multiprocessing.Process(target=master, args=(condition, data1, data2))

    agent1_process.start()
    agent2_process.start()
    master_process.start()

    agent1_process.join()
    agent2_process.join()
    master_process.join()

    print("Données de l'agent 1:", data1)
    print("Données de l'agent 2:", data2)


In [ ]:
import multiprocessing

# Shared data between processes
shared_data = []

# Create a Condition object
condition = multiprocessing.Condition()

# Function for a process that appends data to the shared list
def append_data():
    global shared_data
    with condition:
        shared_data.append("Data")
        print("Data appended.")
        condition.notify()  # Notify waiting processes

# Function for another process that consumes data from the shared list
def consume_data():
    global shared_data
    with condition:
        if not shared_data:
            print("Waiting for data...")
            condition.wait()  # Wait for data to be appended
        print("Data consumed:", shared_data.pop())

if __name__ == "__main__":
    process1 = multiprocessing.Process(target=append_data)
    process2 = multiprocessing.Process(target=consume_data)

    process1.start()
    process2.start()

    process1.join()
    process2.join()

In [4]:
import multiprocessing

q = multiprocessing.Queue()

q.put("a")
q.put("x")

In [7]:
type(q.qsize())

int